# 🏨 JSON 데이터 언패킹 실습 과제 (호텔스닷컴 리뷰 데이터 정리)

이 노트북은 **원본 JSON 파일을 로딩**한 뒤, 중첩 JSON에서 **호텔 정보 / 리뷰 정보**를 추출하여 `pandas.DataFrame`으로 만들고, 각각 CSV로 저장하는 실습 과제입니다.

---

## 🎯 목표

1. 호텔 단위 정보 → `hotel_info_df` → `호텔정보.csv`
2. 리뷰 단위 정보 → `reviews_df` → `호텔스닷컴리뷰.csv`

---

## ✅ 추출 항목

### 1) 호텔 정보 (호텔 1개당 1행)

| 컬럼명 | 설명 |
|---|---|
| `hotel_id` | 호텔 고유 ID |
| `hotel_name` | 호텔 이름 |
| `hotel_grade` | 호텔 등급 |
| `n_reviews` | 전체 리뷰 수 |
| `avg_rating` | 평균 평점 |
| `cleanliness` | 청결도 평점(없으면 0.0) |
| `service` | 서비스 평점(없으면 0.0) |
| `comfort` | 편안함 평점(없으면 0.0) |
| `condition` | 시설 상태 평점(없으면 0.0) |
| `neighbourhood` | 주변 환경 평점(없으면 0.0) |
| `locality` | 시/도 |
| `region` | 지역(없으면 locality 사용) |

### 2) 리뷰 정보 (리뷰 1개당 1행)

| 컬럼명 | 설명 |
|---|---|
| `hotel_id` | 호텔 고유 ID (호텔 정보에서 가져와서 리뷰에 붙이기) |
| `reviewDate` | 리뷰 작성일 |
| `itineraryId` | 예약(여정) ID |
| `tripType` | 여행 유형 |
| `rating` | 리뷰 평점 |
| `description` | 리뷰 내용 |

---

## ⚠️ 구현 조건

- `for` 반복문으로 호텔/리뷰를 순회할 것
- 중첩 dict 접근을 정확히 수행할 것
- 누락 키는 `.get()`으로 예외 처리할 것
- 최종 결과는 CSV로 저장할 것

---

> **제출물**
> - `호텔정보.csv`
> - `호텔스닷컴리뷰.csv`


## 0. JSON 파일 로딩 (필수)

아래 코드 셀에서 **원본 JSON 파일 경로**를 지정하고 데이터를 로딩하세요.

- 파일이 JSON list 구조라면: `data = json.load(f)` 형태
- 파일이 줄(jsonl) 형태라면: 한 줄씩 읽어 list로 만들기

> ✅ 이 과제에서는 기본적으로 `json.load()`로 로딩 가능한 형태를 가정합니다.


In [6]:
import json
import pandas as pd

[호텔스 닷컴 리뷰 데이터 다운받기 << 클릭](https://drive.google.com/file/d/1yU4U1i3e_SEN6UGYL8iWV-6zDQ76Jkgz/view?usp=sharing)  
파일을 다운받아 data 폴더에 저장하세요.

In [7]:
with open("./data/hotels_korea_reviews_text.json", encoding="utf-8") as file:
    temp = file.read()
    data = json.loads(temp)

## 1. 데이터 구조 확인 (권장)

아래를 실행해서 `data[0]`의 구조를 확인하세요.
- 최상단이 list인지?
- 각 원소가 dict인지?
- `'data' → 'body'` 아래에 무엇이 있는지?
- [JSON formatter 사이트를 활용하세요 <<](https://jsonformatter.curiousconcept.com/#)

- json formatter 사이트에 자료 붙여넣고 구조 확인하기

In [8]:
# 구조 확인 (일부만)
data[0].keys()

dict_keys(['data'])

In [9]:
#data[0]

In [10]:
len(data)

9027

## 2. (힌트) JSON 경로

### 호텔 ID
```python
hotel['data']['body']['pdpHeader']['hotelId']
```

### 리뷰 목록(호텔마다 여러 개)
```python
hotel['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items']
```


## 3. 문제 1) 호텔 정보 DataFrame 만들기

각 호텔(= `for hotel in data`)에 대해 아래 컬럼을 추출하여 `hotel_info_df`를 만드세요.

- 컬럼:  
`hotel_id, hotel_name, hotel_grade, n_reviews, avg_rating, cleanliness, service, comfort, condition, neighbourhood, region, locality`

> ⚠️ `ratingAspects`에 일부 키가 없으면 0.0을 사용하세요.  
> ⚠️ `region`이 없으면 `locality`로 대체하세요.


In [11]:
#data[0] #페이지 9000개 넘게 있으므로 1번째 페이지 먼저 불러옴

In [12]:
data[9]['data']['body']['reviewContent']['overall']['ratingAspects']

# data[9]['data']['body']['reviewContent'] -> n_reviews, avg_rating, cleanliness, service, comfort, condition, neighbourhood
# 이해안됨

{'cleanliness': '10.0', 'service': '10.0', 'condition': '10.0'}

- comfort라는 키 빠져있어서 에러남 -> 키 에러

-> .get() : 있으면 처리, 없으면 뒤의 값(0.0)으로 

- 표에서 중복되는 행 많음
-> setdefault 에서 hotel_id 중복되면 한번 더 쓰지 말고 건너뛰기

In [13]:
hotel_info={} #최종적으로 DataFrame으로 바꿀 딕셔너리

# enumerate = 반복문에서 인덱스랑 값을 같이 줌
# hotel : 실제 호텔 데이터 (dict) / idx : 몇 번째 호텔인지 번호
# idx를 쓰는 이유 → hotel_id 중복 행 제거할 때 필요함

for idx, hotel in enumerate(data):
    #호텔 정보 추출 - json fomatter 구조 보고 쓰기!!
    #hotel_id
    hotel_id=hotel['data']['body']['pdpHeader']['hotelId']

    #hotel_name 
    hotel_name=hotel['data']['body']['propertyDescription']['name']
    #hotel_grade
    hotel_grade=hotel['data']['body']['propertyDescription']['starRating']

    #n_reviews 리뷰 수
    n_reviews=hotel['data']['body']['reviewContent']['overall']['totalCount']
    #평점 avg_rating
    avg_rating=hotel['data']['body']['reviewContent']['overall']['rating']

    #세부 평점 'ratingAspects' 안에 항상 모든 키가 있지 않음
    # 문제에서 ->ratingAspects에 일부 키가 없으면 0.0을 사용하세요.라고 함
    #세부 평점 cleanliness
    cleanliness=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('cleanliness', 0.0))
    #세부 평점 service
    service=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('service',0.0))
    #세부 평점 comfort
    comfort=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('comfort',0.0))
    #세부 평점 condition
    condition=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('condition',0.0))
    #세부 평점 neighbourhood
    neighbourhood=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('neighbourhood',0.0))

    # 문제에서 -> region이 없으면 locality로 대체하라고 했으므로
    #locality를 region보다 먼저 씀.
    #시도 locality
    locality=hotel['data']['body']['propertyDescription']['address']['locality']
    #없을 수 있는 값 -> .get(…, 기본값) 처리
    #지역 region - .get('region', locality) : region 있으면 → region / 없으면 → locality(기본값)
    region=hotel['data']['body']['propertyDescription']['address'].get('region', locality)

    #컬럼명과 값 묶기 - 매칭됨
    #아래에서 zip으로 한 번에 처리 가능
    keys=("hotel_id", "hotel_name", "hotel_grade", 'n_reviews', "avg_rating", "cleanliness",
      "service", "comfort", "condition", "neighbourhood", "region", "locality")
    
    values= (hotel_id, hotel_name, hotel_grade, n_reviews, avg_rating, cleanliness,
      service, comfort, condition, neighbourhood, region, locality)
    
    # 중복 행 제거
    # 중복 hotel_id 제거 (일단 하나 만들고 0번째가 아닌 것부터 필터링)
    # 첫 번째 호텔(idx == 0)는 무조건 저장
    # 하지만 그 다음 인덱스부터 hotel_id가 이미 hotel_info['hotel_id']에 있으면 break
    for key, value in zip(keys, values):
        if idx>0 and key== 'hotel_id':
            if value in hotel_info[key]:
                break
        hotel_info.setdefault(key,[]).append(value)
        #setdefault: key가 없으면 → 빈 리스트 만들고 / 있으면 → 기존 리스트에 추가

hotel_info
hotel_info_df=pd.DataFrame(hotel_info) #데이터 프레임(표)으로 변환
hotel_info_df


,hotel_id,hotel_name,hotel_grade,n_reviews,avg_rating,cleanliness,service,comfort,condition,neighbourhood,region,locality
0,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시
1,589827,바다마루 제주 (Badamaru Jeju),2.5,20,8.4,9.0,8.4,8.6,8.4,5.0,제주특별자치도,서귀포시
2,589840,아리수 호텔 (Hotel Arisu),3.0,31,8.2,8.2,8.4,8.2,8.2,6.0,경기도,김포시
3,1138393120,힐링모텔 (Healing Motel),2.5,10,6.2,6.2,6.2,7.0,6.2,8.0,인천광역시,인천광역시
4,1366851616,동대문 PR 모던 스튜디오 (Dongdaemun PR. Mondern Studio),3.0,1,8.0,10.0,10.0,0.0,10.0,0.0,서울특별시,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...
2832,474897,스타호스텔 명동 (Star Hostel Myeongdong Ing),2.0,14,4.8,3.4,5.8,4.2,4.2,8.0,서울특별시,서울특별시
2833,507666,해와돌바라기 (Jeju Baraki),3.0,18,8.6,9.0,9.4,8.8,8.8,8.0,제주특별자치도,서귀포시
2834,507669,호텔 토스카나 (Hotel Toscana),4.0,608,9.2,9.2,9.2,9.4,9.2,7.8,제주특별자치도,서귀포시
2835,718356256,평창펜션하늘숲 (Pyeongchang The Sky Forest),3.0,3,7.4,8.6,8.0,6.6,7.4,6.0,강원도,평창군


In [14]:
#최종 정리 - 호텔 정보

hotel_info={}

for idx, hotel in enumerate(data):
    #hotel_id
    hotel_id=hotel['data']['body']['pdpHeader']['hotelId']
    #hotel_name 
    hotel_name=hotel['data']['body']['propertyDescription']['name']
    #hotel_grade
    hotel_grade=hotel['data']['body']['propertyDescription']['starRating']
    #n_reviews 리뷰 수
    n_reviews=hotel['data']['body']['reviewContent']['overall']['totalCount']
    #평점 avg_rating
    avg_rating=hotel['data']['body']['reviewContent']['overall']['rating']
    #세부 평점 cleanliness
    cleanliness=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('cleanliness', 0.0))
    #세부 평점 service
    service=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('service',0.0))
    #세부 평점 comfort
    comfort=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('comfort',0.0))
    #세부 평점 condition
    condition=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('condition',0.0))
    #세부 평점 neighbourhood
    neighbourhood=float(hotel['data']['body']['reviewContent']['overall']['ratingAspects'].get('neighbourhood',0.0))
    #시도 locality
    locality=hotel['data']['body']['propertyDescription']['address']['locality']
    #지역 region
    region=hotel['data']['body']['propertyDescription']['address'].get('region', locality)


    keys=("hotel_id", "hotel_name", "hotel_grade", 'n_reviews', "avg_rating", "cleanliness",
      "service", "comfort", "condition", "neighbourhood", "region", "locality")
    
    values= (hotel_id, hotel_name, hotel_grade, n_reviews, avg_rating, cleanliness,
      service, comfort, condition, neighbourhood, region, locality)
    
    for key, value in zip(keys, values):
        if idx>0 and key== 'hotel_id':
            if value in hotel_info[key]:
                break
        hotel_info.setdefault(key,[]).append(value)

hotel_info
hotel_info_df=pd.DataFrame(hotel_info)
hotel_info_df


,hotel_id,hotel_name,hotel_grade,n_reviews,avg_rating,cleanliness,service,comfort,condition,neighbourhood,region,locality
0,1041793024,상상속의집 (SangSang Hotel),3.0,67,7.8,8.0,7.6,8.2,7.8,8.2,경상남도,거제시
1,589827,바다마루 제주 (Badamaru Jeju),2.5,20,8.4,9.0,8.4,8.6,8.4,5.0,제주특별자치도,서귀포시
2,589840,아리수 호텔 (Hotel Arisu),3.0,31,8.2,8.2,8.4,8.2,8.2,6.0,경기도,김포시
3,1138393120,힐링모텔 (Healing Motel),2.5,10,6.2,6.2,6.2,7.0,6.2,8.0,인천광역시,인천광역시
4,1366851616,동대문 PR 모던 스튜디오 (Dongdaemun PR. Mondern Studio),3.0,1,8.0,10.0,10.0,0.0,10.0,0.0,서울특별시,서울특별시
...,...,...,...,...,...,...,...,...,...,...,...,...
2832,474897,스타호스텔 명동 (Star Hostel Myeongdong Ing),2.0,14,4.8,3.4,5.8,4.2,4.2,8.0,서울특별시,서울특별시
2833,507666,해와돌바라기 (Jeju Baraki),3.0,18,8.6,9.0,9.4,8.8,8.8,8.0,제주특별자치도,서귀포시
2834,507669,호텔 토스카나 (Hotel Toscana),4.0,608,9.2,9.2,9.2,9.4,9.2,7.8,제주특별자치도,서귀포시
2835,718356256,평창펜션하늘숲 (Pyeongchang The Sky Forest),3.0,3,7.4,8.6,8.0,6.6,7.4,6.0,강원도,평창군


### 3-1) 호텔 정보 CSV 저장

- 파일명: `호텔정보.csv`
- 인덱스 저장하지 않기(`index=False`)


In [15]:
hotel_info_df.to_csv("./data/호텔정보.csv",index=False,)

## 4. 문제 2) 리뷰 정보 DataFrame 만들기

각 호텔에서 리뷰 목록을 찾아 **모든 리뷰를 하나로 합친 DataFrame**을 만들고, `hotel_id`를 함께 저장하세요.

- 컬럼:  
`hotel_id, reviewDate, itineraryId, tripType, rating, description`

> ⚠️ 리뷰 목록이 없을 수도 있으니 예외 처리를 고려하세요(예: `.get()` 또는 `try/except`).


In [16]:
#hotel_id 
hotel_id=data[0]['data']['body']['pdpHeader']['hotelId']
hotel_id

1041793024

In [17]:
# 사용자 리뷰 리스트
data[0]['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items']
#reviews=data[0]['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items']

#개수
len(data[0]['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items'])

42

In [18]:
# 첫 번째 리뷰
data[0]['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items'][0]

{'itineraryId': '72009037635503',
 'brand': 'expedia',
 'genuineMsg': 'Expedia 실제 이용 고객 후기',
 'googleTranslateEnabled': False,
 'tripType': 'family',
 'tripTypeText': '1박 가족 여행',
 'reviewDate': '2020년 11월 20일',
 'reviewDbDate': '1606003200000',
 'reviewer': {'name': '나미', 'locality': 'kr', 'locale': 'ko_KR'},
 'rating': '10.0',
 'badge': '최고 좋음',
 'summary': '',
 'description': '주차가 객실당 한칸씩 차고가 있는게 굉장히 특이했으며 가장 편리했음.\n간혹 차에 무언가를 두고와서 가지러 가는 경험을 하는데 리조트나 다른 펜션이었다면 주차장까지 내려가는 번거로움이 있었겠지만 이 호텔에서는 객실 바로밑에 주차장이 있어서 주차장이 가장편리 하였다.\n하지만 방음 수준은 최악'}

In [23]:
# 사용자 리뷰 리스트
#print(data[0]['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items'])
#reviews=data[0]['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items']

for idx, review in enumerate(reviews):
    # hotel_id, reviewDate, itineraryId, tripType, rating, description
    #reviewDate
    # for문 씌워줘서
    #reviewDate= data[0]['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items'][0]['reviewDate']
    reviewDate= review['reviewDate']
    #itineraryId
    itineraryId= review['itineraryId']
    #tripType
    tripType= review['tripType']
    #rating
    rating =review['rating']
    #description
    description= review['description']
    
    #print(reviewDate, itineraryId, tripType, rating, description)

In [20]:
import re

In [21]:
# 사용자 리뷰 리스트
user_reviews={}

# 호텔 단위로 반복 : hotel → 한 호텔의 전체 데이터 / idx → 몇 번째 호텔인지 (진행상황 출력용)
for idx, hotel in enumerate(data):
    print(f"{idx}/{len(data)-1} 수집중 ",end='\r')
    if len(hotel['data']['body']['reviewContent']['reviews']['hermes']['groups'])!=0:
        reviews=hotel['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items']
        for idx2, review in enumerate(reviews):
            hotel_id=hotel['data']['body']['pdpHeader']['hotelId']
            reviewDate= review['reviewDate']
            itineraryId= review['itineraryId']
            tripType= review['tripType']
            rating =float(review['rating'])
            description= re.sub("[^가-힣a-zA-Z0-9]", " ", review['description'].replace("\n"," ").replace("  "," ").strip())

            keys=("hotel_id", "reviewDate", 'itineraryId', 'tripType', 'rating', 'description')
            values=(hotel_id, reviewDate, itineraryId, tripType, rating, description)

            for key, value in zip(keys, values):
                user_reviews.setdefault(key,[]).append(value)

user_reviews_df=pd.DataFrame(user_reviews)
user_reviews_df

,hotel_id,reviewDate,itineraryId,tripType,rating,description
0,1041793024,2020년 11월 20일,72009037635503,family,10.0,주차가 객실당 한칸씩 차고가 있는게 굉장히 특이했으며 가장 편리했음 간혹 차에 무...
1,1041793024,2020년 11월 14일,9202656047317,family,10.0,1 오션뷰 남해안의 어느 숙박업소가 그렇듯이 오션뷰가 좋습니다 이 호텔앞의...
2,1041793024,2020년 8월 11일,9196559102828,other,10.0,커플이 다녀가기에 좋은 숙소 프라이빗함과 뷰도 갖춘 좋은 곳
3,1041793024,2020년 8월 10일,9201554317978,other,8.0,위치와 뷰는 정말 좋습니다 개별주차장이 생각보다 쓰기 좋았습니다 다만 호텔보다...
4,1041793024,2020년 8월 7일,9196473456283,family,8.0,스탠다드 이용했어요 방마다 차고가 있어서 편하게 짐 옮겼네요 제가 이용한 방에 티비...
...,...,...,...,...,...,...
171630,507669,2018년 1월 7일,8106039263663,other,8.0,
171631,718356256,2018년 8월 4일,8120843372947,family,8.0,사진을 굉장히 잘 찍으신듯 해요 4명이 이용 가능한 펜션 알아보다 방문했는데 사...
171632,1344945952,2020년 9월 14일,9196902905665,business,8.0,
171633,1344945952,2020년 7월 13일,9201269638204,business,2.0,


In [21]:
# 사용자 리뷰 리스트 최종
user_reviews={}

for idx, hotel in enumerate(data):
    print(f"{idx}/{len(data)-1} 수집중 ",end='\r')
    if len(hotel['data']['body']['reviewContent']['reviews']['hermes']['groups'])!=0:
        reviews=hotel['data']['body']['reviewContent']['reviews']['hermes']['groups'][0]['items']
        for idx2, review in enumerate(reviews):
            hotel_id=hotel['data']['body']['pdpHeader']['hotelId']
            reviewDate= review['reviewDate']
            itineraryId= review['itineraryId']
            tripType= review['tripType']
            rating =float(review['rating'])
            description= re.sub("[^가-힣a-zA-Z0-9]", " ", review['description'].replace("\n"," ").replace("  "," ").strip())

            keys=("hotel_id", "reviewDate", 'itineraryId', 'tripType', 'rating', 'description')
            values=(hotel_id, reviewDate, itineraryId, tripType, rating, description)

            for key, value in zip(keys, values):
                user_reviews.setdefault(key,[]).append(value)

user_reviews_df=pd.DataFrame(user_reviews)
user_reviews_df

,hotel_id,reviewDate,itineraryId,tripType,rating,description
0,1041793024,2020년 11월 20일,72009037635503,family,10.0,주차가 객실당 한칸씩 차고가 있는게 굉장히 특이했으며 가장 편리했음 간혹 차에 무...
1,1041793024,2020년 11월 14일,9202656047317,family,10.0,1 오션뷰 남해안의 어느 숙박업소가 그렇듯이 오션뷰가 좋습니다 이 호텔앞의...
2,1041793024,2020년 8월 11일,9196559102828,other,10.0,커플이 다녀가기에 좋은 숙소 프라이빗함과 뷰도 갖춘 좋은 곳
3,1041793024,2020년 8월 10일,9201554317978,other,8.0,위치와 뷰는 정말 좋습니다 개별주차장이 생각보다 쓰기 좋았습니다 다만 호텔보다...
4,1041793024,2020년 8월 7일,9196473456283,family,8.0,스탠다드 이용했어요 방마다 차고가 있어서 편하게 짐 옮겼네요 제가 이용한 방에 티비...
...,...,...,...,...,...,...
171630,507669,2018년 1월 7일,8106039263663,other,8.0,
171631,718356256,2018년 8월 4일,8120843372947,family,8.0,사진을 굉장히 잘 찍으신듯 해요 4명이 이용 가능한 펜션 알아보다 방문했는데 사...
171632,1344945952,2020년 9월 14일,9196902905665,business,8.0,
171633,1344945952,2020년 7월 13일,9201269638204,business,2.0,


### 4-1) 리뷰 CSV 저장

- 파일명: `호텔스닷컴리뷰.csv`
- 인덱스 저장하지 않기(`index=False`)


In [22]:
user_reviews_df.to_csv("./data/호텔스닷컴리뷰.csv", index=False)